In [142]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit import Aer
from qiskit.circuit.library import QFT
from copy import deepcopy


from tools.classical import *
from tools.quantum import *

In [143]:
num_qubits = 4
num_estimating_qubits = 6
qpe_precision = 2 ** (-num_estimating_qubits)

X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()

### Find Heisenberg coefficients for non degenerate spectrum

In [144]:
coeff_lower_bound = 0
coeff_xx = np.arange(1, coeff_lower_bound, -0.1)
coeff_yy = np.arange(1, coeff_lower_bound, -0.1)
coeff_zz = np.arange(1, coeff_lower_bound, -0.1)
coeff_z = np.arange(1, coeff_lower_bound, -0.1)

#* Find ideal spectrum
coeffs = np.array(np.meshgrid(coeff_xx, coeff_yy, coeff_zz, coeff_z)).T.reshape(-1, 4)
hamiltonian_ideal_qt = None
coeffs_ideal_spec = []
found_ideal_spectrum = False
for coeff in coeffs:
    hamiltonian_qt = hamiltonian_matrix([X, X], [Y, Y], [Z, Z], coeffs=coeff, num_qubits=num_qubits, symbreak_term=[Z])
    rescaling_factor, shift = rescaling_and_shift_factors(hamiltonian_qt)
    rescaling_factor /= (1 - 2 * qpe_precision)  # [0, 1 - eps]
    shift = 0. #! For negatives
    rescaled_hamiltonian_qt = hamiltonian_qt / rescaling_factor + shift * qt.qeye(hamiltonian_qt.shape[0])
    
    rescaled_exact_spec = np.linalg.eigvalsh(rescaled_hamiltonian_qt)
    
    # Accept coeff only if all spectrum elements are not closer than qpe_precision
    for eigval_i in rescaled_exact_spec:
        spec_without_eigval_i = np.delete(rescaled_exact_spec, np.where(rescaled_exact_spec == eigval_i))
        if np.any(np.abs(spec_without_eigval_i - eigval_i) < qpe_precision):
            break
    else:
        found_ideal_spectrum = True
        hamiltonian_ideal_qt = hamiltonian_qt
        coeffs_ideal_spec = coeff
        exact_spec = np.linalg.eigvalsh(hamiltonian_qt)
        print('Original spectrum: ', np.round(exact_spec, 4))
        print("Ideal spectrum found: ", np.round(rescaled_exact_spec, 4))
        print("Nonrescaled coefficients: ", coeffs_ideal_spec)
        break

if not found_ideal_spectrum:
    print("No ideal spectrum found")
    
#* Found ideal spectrum with bulk sym break term Z, strength 1, for 3-4 and 4-6 system qubits - estimating qubits
    
rescaling_factor, shift = rescaling_and_shift_factors(hamiltonian_ideal_qt)
rescaling_factor /= (1 - 2 * qpe_precision)  # [min * (1 - eps) / gamma , max * (1 - eps) / gamma] ~ [-4.7, 4.7]
shift = 0. #! For negatives
print(f'Rescaling factor {rescaling_factor}, shift {shift}')

rescaled_hamiltonian_qt = hamiltonian_qt / rescaling_factor + shift * qt.qeye(hamiltonian_qt.shape[0])  #* Shift / No Shift
rescaled_coeff = coeffs_ideal_spec / rescaling_factor
print('Rescaled coefficients: ', rescaled_coeff)


Original spectrum:  [-7.0747 -4.7382 -4.     -2.2702 -1.5231 -1.2967 -0.6882  0.      0.6882
  1.0362  1.2967  1.8451  2.2702  3.5419  4.7382  6.1746]
Ideal spectrum found:  [-0.5173 -0.3464 -0.2925 -0.166  -0.1114 -0.0948 -0.0503 -0.      0.0503
  0.0758  0.0948  0.1349  0.166   0.259   0.3464  0.4515]
Nonrescaled coefficients:  [1.  0.6 1.  1. ]
Rescaling factor 13.676665806451613, shift 0.0
Rescaled coefficients:  [0.07311724 0.04387034 0.07311724 0.07311724]


In [145]:
np.random.seed(666)
jumped = False
exact_spec, eigenstates = np.linalg.eigh(rescaled_hamiltonian_qt)

#* Initial state = eigenstate
eig_index = 0
initial_state = eigenstates[:, eig_index]

#* Jump
# jumped = True
# initial_energy = initial_state.conj().T @ rescaled_hamiltonian_qt.full() @ initial_state

# rand_qubit = np.random.randint(num_qubits)
# print(f'Jump on qubit {rand_qubit}')

# jump_op = pad_term([X], num_qubits, rand_qubit)
# state_after_jump = jump_op.full() @ initial_state
# energy_after_jump = state_after_jump.conj().T @ rescaled_hamiltonian_qt.full() @ state_after_jump  # After jump energy
# print(f'Energy after jump = {energy_after_jump}')

#* Initial state = superposition of two eigenstates
# initial_state = np.sqrt(0.6) * eigenstates[:, 3] + np.sqrt(0.4) * eigenstates[:, 4]
#TODO: try complex states / amplitudes

#* Initial state = random state
# randstate_better = np.zeros(2**num_qubits)
# randstate_better[np.random.choice(2**num_qubits, 2**num_qubits//2, replace=False)] = 1
# randstate_better /= np.linalg.norm(randstate_better)
# initial_state = randstate_better

initial_energy = initial_state.conj().T @ rescaled_hamiltonian_qt.full() @ initial_state
print(f'Initial energy = {initial_energy}')

#* Initialize
initial_state = Statevector(initial_state)
qr_energy = QuantumRegister(num_estimating_qubits, name="E")
cr_energy = ClassicalRegister(num_estimating_qubits, name="crE")
qr_sys  = QuantumRegister(num_qubits, name="sys")
circ = QuantumCircuit(qr_energy, qr_sys, cr_energy)
circ.initialize(initial_state, qr_sys)
circ.h(qr_energy)

# Jump
if jumped:
    circ.x(qr_sys[rand_qubit])

statevector = Statevector(circ).data
padded_rescaled_hamiltonian = np.kron(rescaled_hamiltonian_qt.full(), np.eye(2**num_estimating_qubits))  # top-bottom = right-left
energy = statevector.conj().T @ padded_rescaled_hamiltonian @ statevector
print(f'Initial energy in circuit = {energy}')

# QPE
T = 1
time = T * 2 * np.pi
num_trotter_steps = 10

trotter_step_circ = trotter_step_heisenberg(num_qubits, coeffs=rescaled_coeff, symbreak=True)
U = ham_evol(num_qubits, trotter_step=trotter_step_circ, num_trotter_steps=num_trotter_steps, time=time)
signU = ham_evol(num_qubits, trotter_step=trotter_step_circ, num_trotter_steps=num_trotter_steps, time=-time)  #! Negative time
cU = U.control(1, label='+')
csignU = signU.control(1, label='-')
for q in range(num_estimating_qubits):
    circ.p(time * shift * 2**q, qr_energy[q])  #* Shift
    if q != num_estimating_qubits - 1:
        for _ in range(2**q):
            circ.compose(cU, [q, *list(qr_sys)], inplace=True)
    else:  # q = last qubit (MSB) is the sign qubit
        for _ in range(2**q):
            circ.compose(csignU, [q, *list(qr_sys)], inplace=True)

# Inverse QFT
qft_circ = QFT(num_estimating_qubits, inverse=True)  # With SWAPs
circ.compose(qft_circ, qr_energy, inplace=True)

circ.measure(qr_energy, cr_energy)
print(circ)

Jump on qubit 0
Initial energy = (-0.2924689435719857+0j)
Initial energy in circuit = (-0.2924689435719851+0j)


                                   ┌───┐                            ┌──────┐»
  E_0: ────────────────────────────┤ H ├────────────────────────────┤ P(0) ├»
                                   ├───┤                            ├──────┤»
  E_1: ────────────────────────────┤ H ├────────────────────────────┤ P(0) ├»
                                   ├───┤                            ├──────┤»
  E_2: ────────────────────────────┤ H ├────────────────────────────┤ P(0) ├»
                                   ├───┤                            ├──────┤»
  E_3: ────────────────────────────┤ H ├────────────────────────────┤ P(0) ├»
                                   ├───┤                            ├──────┤»
  E_4: ────────────────────────────┤ H ├────────────────────────────┤ P(0) ├»
                                   ├───┤                            ├──────┤»
  E_5: ────────────────────────────┤ H ├────────────────────────────┤ P(0) ├»
       ┌───────────────────────────┴───┴────────────────────────

In [146]:
tr_circ = transpile(circ, basis_gates=['u', 'cx'], optimization_level=3)

In [147]:
simulator = Aer.get_backend('statevector_simulator')
shots = 1000
job = simulator.run(tr_circ, shots=shots)
counts = job.result().get_counts()
counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))

print(counts)


{'101101': 693, '101110': 152, '101100': 45, '101111': 27, '101011': 15, '110000': 10, '101010': 9, '110001': 8, '110100': 4, '111001': 4, '100110': 3, '101001': 3, '110011': 3, '100101': 2, '100000': 2, '110010': 2, '110101': 2, '010010': 2, '101000': 2, '001101': 1, '100100': 1, '111010': 1, '110110': 1, '011101': 1, '000100': 1, '010000': 1, '100111': 1, '011010': 1, '001010': 1, '010011': 1, '000000': 1}


In [148]:
phase_bits = list(counts.keys())[0] # take the most often obtaned result
phase_bits_shots = counts[phase_bits]
#* Main bitstring result
# signed binary to decimal:
if phase_bits[0] == '1':
    phase = (int(phase_bits[1:], 2) - 2**(num_estimating_qubits - 1)) / 2**num_estimating_qubits  # exp(i 2pi phase)
else:
    phase = int(phase_bits[1:], 2) / 2**num_estimating_qubits

#* Combine phases
combined_phase = 0.
# Combine all phases
for i in range(len(counts.keys())):
    if list(counts.keys())[i][0] == '1':
        phase_part = (int(list(counts.keys())[i][1:], 2) - 2**(num_estimating_qubits - 1)) / 2**num_estimating_qubits
    else:
        phase_part = int(list(counts.keys())[i][1:], 2) / 2**num_estimating_qubits
        
    combined_phase += phase_part * list(counts.values())[i] / shots

estimated_energy = phase / T  # exp(i 2pi phase) = exp(i 2pi E T)
estimated_combined_energy = combined_phase / T
if jumped:
    print(f'Exact energy: {energy_after_jump.real}')
else:
    print(f'Exact energy: {initial_energy.real}')
print(f'Estimated energy: {estimated_energy}')  # I guess it peaks at the two most probable eigenstates and it will give either one of them and
                                                # not the energy in between them.
print(f'Combined estimated energy: {estimated_combined_energy}')  

Exact energy: -0.2924689435719857
Estimated energy: -0.296875
Combined estimated energy: -0.28826562499999986
